## ERDDAPY
* Code taken from [erddapy griddap example](https://github.com/ioos/erddapy/blob/main/notebooks/01a-griddap.ipynb)
* Example uses [PacIOOS ERDDAP](https://pae-paha.pacioos.hawaii.edu/erddap/info/index.html?page=1&itemsPerPage=1000) and [etopo5](https://pae-paha.pacioos.hawaii.edu/erddap/griddap/etopo5.html) as the dataset
  * **NOTE:** the example code given in the github dataset works

In [1]:
from erddapy import ERDDAP

e = ERDDAP(
    server="https://comet.nefsc.noaa.gov/erddap",
    protocol="griddap",
)

e.dataset_id = "occci_v6_daily_1km"

In [2]:
import json

e.griddap_initialize()

print(f"variables in this dataset:\n\n{e.variables}")
constraints = json.dumps(e.constraints, indent=1)
print(f"\nconstraints of this dataset:\n\n{constraints}")

e.variables = [e.variables[15]]

print(f"Downloading {e.variables}.")

variables in this dataset:

['MERIS_nobs', 'MODISA_nobs', 'OLCI_A_nobs', 'OLCI_B_nobs', 'Rrs_412', 'Rrs_443', 'Rrs_490', 'Rrs_510', 'Rrs_560', 'Rrs_665', 'SeaWiFS_nobs', 'VIIRS_nobs', 'adg_443', 'aph_443', 'atot_443', 'chlor_a', 'kd_490', 'total_nobs', 'water_class1', 'water_class10', 'water_class11', 'water_class12', 'water_class13', 'water_class14', 'water_class2', 'water_class3', 'water_class4', 'water_class5', 'water_class6', 'water_class7', 'water_class8', 'water_class9']

constraints of this dataset:

{
 "time>=": "2025-01-24T00:00:00Z",
 "time<=": "2025-01-24T00:00:00Z",
 "time_step": 1,
 "latitude>=": 89.99479166666667,
 "latitude<=": -89.99479166666666,
 "latitude_step": 1,
 "longitude>=": -179.99479166666666,
 "longitude<=": 179.99479166666663,
 "longitude_step": 1
}


In [3]:
import pandas as pd
def bounds2contraints(bounds):
    return {
        "time>=": bounds.datemin.squeeze(),
        "time<=": bounds.datemax.squeeze(),
        "longitude>=": bounds.miny.squeeze(),
        "longitude<=": bounds.maxy.squeeze(),
        "latitude>=": bounds.minx.squeeze(),
        "latitude<=": bounds.maxx.squeeze(),
    }


e.constraints.update(bounds2contraints(pd.DataFrame({'datemin':["2024-07-01T00:00:00Z"],'datemax':["2024-07-10T00:00:00Z"],'minx':[34],'miny':[-77],'maxx':[46],'maxy':[-63]})))
e.constraints

{'time>=': '2024-07-01T00:00:00Z',
 'time<=': '2024-07-10T00:00:00Z',
 'time_step': 1,
 'latitude>=': 34,
 'latitude<=': 46,
 'latitude_step': 1,
 'longitude>=': -77,
 'longitude<=': -63,
 'longitude_step': 1}

In [4]:
ds = e.to_xarray()

HTTPError: <!doctype html><html lang="en"><head><title>HTTP Status 400 – Bad Request</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 400 – Bad Request</h1><hr class="line" /><p><b>Type</b> Exception Report</p><p><b>Message</b> Invalid character found in the request target [&#47;erddap&#47;griddap&#47;occci_v6_daily_1km.nc?chlor_a[(2024-07-01T00:00:00Z):1:(2024-07-10T00:00:00Z)][(34):1:(46)][(-77):1:(-63)] ]. The valid characters are defined in RFC 7230 and RFC 3986</p><p><b>Description</b> The server cannot or will not process the request due to something that is perceived to be a client error (e.g., malformed request syntax, invalid request message framing, or deceptive request routing).</p><p><b>Exception</b></p><pr

I THINK the error is caused by an incorrect URL... 
* The URL that the package is trying to pull from is: https://comet.nefsc.noaa.gov/erddap/griddap/occci_v6_daily_1km.nc?chlor_a[(2024-07-01T00:00:00Z):1:(2024-07-10T00:00:00Z)][(34):1:(46)][(-77):1:(-63)]
<br>

* A working link (created in the ERDDAP webpage) is: https://comet.nefsc.noaa.gov/erddap/griddap/occci_v6_daily_1km.nc?chlor_a%5B(2024-07-01T00:00:00Z):1:(2024-07-10T00:00:00Z)%5D%5B(34):1:(46)%5D%5B(-77):1:(-63)%5D

<br>
I've tested several other erddaps with the same results